In [225]:
import json
import requests
import io
import base64
from PIL import Image, PngImagePlugin, ImageDraw
import os
from datetime import datetime


In [183]:
os.startfile('webui-user.lnk')
# os.startfile('webui-user.bat')

In [184]:
masks_directory = 'masks/'
result_photos_directory = 'images/'
source_photos_directory = 'source/'

if not os.path.exists(masks_directory):
    os.makedirs(masks_directory)

if not os.path.exists(result_photos_directory):
    os.makedirs(result_photos_directory)

if not os.path.exists(source_photos_directory):
    os.makedirs(source_photos_directory)

In [185]:
def toB64(imgPath):
    with open(imgPath, "rb") as imageFile:
        return base64.b64encode(imageFile.read()).decode("utf-8")

In [186]:
photo_path = source_photos_directory + 'photo_front.jpg'
img = toB64(photo_path)

In [265]:
with Image.open(photo_path) as mask_img:
    mask_img.load()
    draw = ImageDraw.Draw(mask_img)
    draw.rectangle((0, mask_img.size[1] - 550, mask_img.size[0], mask_img.size[1]), fill=(255, 255, 255))
    mask_img = mask_img.convert('L')
    mask_img = mask_img.point(lambda x: 255 if x > 254 else 0)
    mask_img = mask_img.convert('RGBA')

    pixels = mask_img.load()  # create the pixel map

    for i in range(mask_img.size[0]):  # for every pixel:
        for j in range(mask_img.size[1]):
            if pixels[i, j] == (0, 0, 0, 255):
                pixels[i, j] = (0, 0, 0, 0)
            # if pixels[i, j] == (255, 255, 255, 255):
            #     pixels[i, j] = (255, 255, 255, 77)

    mask_img.save(masks_directory + 'mask.png')

In [266]:
mask_path = masks_directory + 'mask.png'
mask = toB64(mask_path)

In [270]:
url = "http://127.0.0.1:7860"

payload_img2img = {
    "prompt": "black business suit",  # Текстовая строка, описывающая изображение, которое вы хотите создать

    "negative_prompt": 'red color',  # Текстовое описание того, что вы не хотите видеть на изображении.

    "steps": 50,  # Количество шагов шумоподавления, используемых моделью ИИ при создании изображения из текстовой подсказки. Более высокое значение приводит к увеличению времени генерации и не обязательно гарантирует более высокое качество изображения.

    'sampler_name': 'DPM++ 2M SDE Karras',  #
    # 'sampler_name': 'Euler',  # Имя метода сэмплера, который будет использоваться для создания изображения. Вывод изображения немного отличается в зависимости от метода.

    'include_init_images': False,  # Установите значение true, если вы планируете включить начальные изображения в полезные данные или если вы хотите выполнить рисование на изображении.

    "init_images": [img],  # Список изображений в кодировке base64, которые будут использоваться для генерации

    "width": 512,  # Ширина изображения, которое вы хотите создать.

    "height": 512,  # Высота изображения, которое требуется создать.

    'inpainting_mask_invert': 0,  # Определяет, требуется ли закрашивать маскируемый объект или снаружи маскируемого объекта. 0 закрасит замаскированный объект. 1 закрашивает область за пределами маскируемого объекта. Этот параметр необходим для закрашивания.

    'inpainting_fill': 0,  # Определяет, как должно отображаться замаскированное содержимое. Это может быть 0, 1, 2 или 3. 0 -> "заливка" 1 -> "исходный" 2 -> "скрытый шум" 3 -> "скрытое ничего"

    'resize_mode': 0,  # Это может быть 0, 1, 2 или 3. 0 -> "просто изменить размер" 1 -> "обрезать и изменить размер" 2 -> "изменить размер" 3 -> "заливка"

    "inpaint_full_res_padding": 32,  # Заполнение вокруг замаскированного объекта для закрашивания в полном разрешении Этот параметр необходим для закрашивания.

    "inpaint_full_res": True,  # Относится к области закрашивания. Оно может быть как истинным, так и ложным. Если установлено значение true, область закрашивания будет иметь то же разрешение, что и исходное изображение. Если установлено значение false, область закрашивания будет растягиваться в соответствии с начальным изображением. Этот параметр необходим для закрашивания.

    # 'image_cfg_scale': 4,  # 	Определяет, насколько результат похож на исходное изображение, поэтому более низкое значение означает более сильный эффект. Это полная противоположность cfg_scale.

    'cfg_scale': 7,  # Определяет, насколько точно он следует запросу. 7 или 7,5 обычно работает лучше всего. Вы можете увеличить значение, если сгенерированное изображение не соответствует вашему запросу.Низкие числа (0-6): вы сообщаете SD, что она может игнорировать ваше приглашение. Средний диапазон (6-10 баллов): вы говорите SD, что хотели бы, чтобы он делал то, о чем вы просите, но вы не возражаете против небольшого разнообразия.Высокие цифры (10+): мы обнаружили, что высокое количество cfg часто приводит к очень плохому изображению.

    'denoising_strength': 0.8,  # Определяет, насколько близко сгенерированное изображение будет к предоставленному. Чем ближе он к 0, тем ближе он будет к предоставленному изображению.

    'mask': mask,  # Изображение в кодировке Base64, которое будет использоваться для закрашивания. Необходимо указать область исходного изображения, которую вы хотите изменить.

    'mask_blur_x': 4, 'mask_blur_y': 4, 'mask_blur': 14,  #

}

sett = {
    'sd_model_checkpoint': 'Deliberate_v5-inpainting.safetensors [ab70e869ff]',
}

In [272]:
time_key = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

settings = requests.post(url=f'{url}/sdapi/v1/options', json=sett)
response_img2img = requests.post(url=f'{url}/sdapi/v1/img2img', json=payload_img2img)

r_img2img = response_img2img.json()
print(r_img2img['parameters'])

for i in r_img2img['images']:
    image = Image.open(io.BytesIO(base64.b64decode(i.split(",", 1)[0])))

    png_payload = {
        "image": "data:image/png;base64," + i
    }
    response2 = requests.post(url=f'{url}/sdapi/v1/png-info', json=png_payload)

    pnginfo = PngImagePlugin.PngInfo()
    pnginfo.add_text("parameters", response2.json().get("info"))

    image.save(f"{result_photos_directory}{'_'.join(payload_img2img['prompt'].split(' '))}_output_{time_key}.png", pnginfo=pnginfo)


{'prompt': 'black business suit', 'negative_prompt': 'red color', 'styles': None, 'seed': -1, 'subseed': -1, 'subseed_strength': 0, 'seed_resize_from_h': -1, 'seed_resize_from_w': -1, 'sampler_name': 'DPM++ 2M SDE Karras', 'batch_size': 1, 'n_iter': 1, 'steps': 50, 'cfg_scale': 7.0, 'width': 512, 'height': 512, 'restore_faces': None, 'tiling': None, 'do_not_save_samples': False, 'do_not_save_grid': False, 'eta': None, 'denoising_strength': 0.8, 's_min_uncond': None, 's_churn': None, 's_tmax': None, 's_tmin': None, 's_noise': None, 'override_settings': None, 'override_settings_restore_afterwards': True, 'refiner_checkpoint': None, 'refiner_switch_at': None, 'disable_extra_networks': False, 'comments': None, 'init_images': None, 'resize_mode': 0, 'image_cfg_scale': None, 'mask': None, 'mask_blur_x': 4, 'mask_blur_y': 4, 'mask_blur': 14, 'inpainting_fill': 0, 'inpaint_full_res': True, 'inpaint_full_res_padding': 32, 'inpainting_mask_invert': 0, 'initial_noise_multiplier': None, 'latent_ma

In [ ]:
[ссылке](https://www.itshneg.com/kak-ustanovit-stable-diffusion-poshagovaya-instrukciya/?utm_referrer=https%3A%2F%2Fyandex.by%2F)